# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%load_ext autoreload

%autoreload
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import datetime

In [2]:
from create_tables import create_database

In [3]:
create_database()

(<cursor object at 0x7fbaf71c7710; closed: 0>,
 <connection object at 0x7fbaf71533d8; dsn: 'user=student password=xxx dbname=sparkifydb host=127.0.0.1', closed: 0>)

In [4]:
try:
    conn.close()
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()
except:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [6]:
#conn.set_session(autocommit=True)

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
filepath = "data/song_data"
song_files = get_files(filepath)

In [8]:
len(song_files)

72

In [9]:
song_files[0:2]

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json']

In [10]:
list_song_df = []
for f in song_files:
    list_song_df.append(pd.read_json(filepath + f.split("song_data")[-1], lines=True))
df = pd.concat(list_song_df).reset_index(drop=True)
#df.head() 

In [11]:
tuple(list_song_df[0])

('artist_id',
 'artist_latitude',
 'artist_location',
 'artist_longitude',
 'artist_name',
 'duration',
 'num_songs',
 'song_id',
 'title',
 'year')

In [12]:
df.head(2)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [13]:
df.artist_name = df.artist_name.str.upper()
df.title = df.title.str.upper()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [14]:

df["year"] = df["year"].values.astype(int)
df["duration"] = df["duration"].values.astype(float)
song_df = df[["song_id","title","artist_id","year","duration"]]
#song_data = tuple([list(row) for row in song_data.itertuples(index=False)][0])

In [15]:
try: 
    cur.execute(song_table_create)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:
song_df.head()

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I DIDN'T MEAN TO,ARD7TVE1187B99BFB1,0,218.93179
1,SOUDSGM12AC9618304,INSATIABLE (INSTRUMENTAL VERSION),ARNTLGG11E2835DDB9,0,266.39628
2,SOIAZJW12AB01853F1,PINK WORLD,AR8ZCNI1187B9A069B,1984,269.81832
3,SOHKNRJ12A6701D1F8,DROP OF RAIN,AR10USD1187B99F3F1,0,189.57016
4,SOCIWDW12A8C13D406,SOUL DEEP,ARMJAGH1187FB546F3,1969,148.03546


In [17]:
song_table_insert

'INSERT INTO songs (song_id,title,artist_id,year,duration)                  VALUES (%s, %s, %s, %s, %s)'

In [18]:
for row in song_df.itertuples(index=False):
    print(tuple(row))
    cur.execute(song_table_insert, tuple(row))
    conn.commit()

('SOMZWCG12A8C13C480', "I DIDN'T MEAN TO", 'ARD7TVE1187B99BFB1', 0, 218.93179)
('SOUDSGM12AC9618304', 'INSATIABLE (INSTRUMENTAL VERSION)', 'ARNTLGG11E2835DDB9', 0, 266.39628)
('SOIAZJW12AB01853F1', 'PINK WORLD', 'AR8ZCNI1187B9A069B', 1984, 269.81832)
('SOHKNRJ12A6701D1F8', 'DROP OF RAIN', 'AR10USD1187B99F3F1', 0, 189.57016)
('SOCIWDW12A8C13D406', 'SOUL DEEP', 'ARMJAGH1187FB546F3', 1969, 148.03546)
('SOYMRWW12A6D4FAB14', 'THE MOON AND I (ORDINARY DAY ALBUM VERSION)', 'ARKFYS91187B98E58F', 0, 267.7024)
('SOMJBYD12A6D4F8557', 'KEEPIN IT REAL (SKIT)', 'ARD0S291187B9B7BF5', 0, 114.78159)
('SOQHXMF12AB0182363', 'YOUNG BOY BLUES', 'ARGSJW91187B9B1D6B', 0, 218.77506)
('SONHOTT12A8C13493C', 'SOMETHING GIRLS', 'AR7G5I41187FB4CE6C', 1982, 233.40363)
('SOFSOCN12A8C143F5D', 'FACE THE ASHES', 'ARXR32B1187FB57099', 2007, 209.60608)
('SOXVLOJ12AB0189215', 'AMOR DE CABARET', 'ARKRRTF1187B9984DA', 0, 177.47546)
('SOMZWCG12A8C13C480', "I DIDN'T MEAN TO", 'ARD7TVE1187B99BFB1', 0, 218.93179)
('SOUQQEA12A8C

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [19]:
artist_df = df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]]
#artist_data = tuple([list(row) for row in artist_data.itertuples(index=False)][0])

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [20]:
try: 
    cur.execute(artist_table_create)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

In [21]:

for row in artist_df.itertuples(index=False):
    cur.execute(artist_table_insert, tuple(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [22]:
log_filepath = "data/log_data"
log_files = get_files(log_filepath)

In [23]:
log_files[0]

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [24]:
len(log_files)

30

In [25]:
list_log = []
l=[]
for index, f in enumerate(log_files):
    f = pd.read_json(open(log_filepath + log_files[index].split("log_data")[-1],"r"), lines=True)
    l.append(f.shape[0])
    list_log.append(f)

    

In [26]:
sum(l)

8056

In [27]:
len(list_log)

30

In [28]:
list_log[0].head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [29]:
list_log[0].tail(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
386,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
387,Deas Vail,Logged In,Elijah,M,0,Davis,237.68771,free,"Detroit-Warren-Dearborn, MI",PUT,NextSong,1.540772e+12,985,Anything You Say (Unreleased Version),200,1543607664796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",5


In [30]:
log_df = pd.concat(list_log)

In [31]:
log_df.shape

(8056, 18)

In [32]:
log_df.tail(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
144,The Police,Logged In,Chloe,F,5,Cuevas,289.85424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,680,So Lonely,200,1542498954796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
145,Bitter:Sweet,Logged In,Jaleah,F,18,Hayes,160.78322,paid,"San Antonio-New Braunfels, TX",PUT,NextSong,1.541003e+12,113,Take 2 Blue,200,1542499040796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,70


In [33]:
log_df.shape

(8056, 18)

In [34]:

log_df.artist = log_df.artist.str.upper()


In [35]:
log_df["ts"] = log_df["ts"].map(lambda x: datetime.datetime.fromtimestamp(x/1000.0))

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [36]:
df_next_song = log_df[log_df["page"] == "NextSong"]

In [37]:
df_next_song.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,STEPHEN LYNCH,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,MANOWAR,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [38]:
#df_next_song["ts"] = df_next_song["ts"].map(lambda x: datetime.datetime.fromtimestamp(x/1000.0))

In [39]:
time_dict = {}
column_labels = ["song","length","start_time","hour", "day", "week_of_year", "month", "year","weekday"]
for key in column_labels:
    time_dict[key] = []
for index,item in df_next_song.iterrows():
    time_dict["song"].append(item["song"].upper())
    time_dict["length"].append(item["length"])
    row = item["ts"]
    time_dict["start_time"].append(row)
    time_dict["hour"].append(row.hour)
    time_dict["day"].append(row.day)
    time_dict["week_of_year"].append(row.isocalendar()[1])
    time_dict["month"].append(row.month)
    time_dict["year"].append(row.isocalendar()[0])
    time_dict["weekday"].append(row.isocalendar()[2])

In [40]:
df_next_song.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,STEPHEN LYNCH,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,MANOWAR,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [41]:
time_df = pd.DataFrame(time_dict)

In [42]:
time_df["start_time"] = time_df["start_time"].map(lambda x:x.strftime('%Y%m%d_%H%M%S'))

In [43]:
time_df.head()

,song,length,start_time,hour,day,week_of_year,month,year,weekday
0,JIM HENSON'S DEAD,182.85669,20181130_002207,0,30,48,11,2018,5
1,SHELL SHOCK,247.56200,20181130_010841,1,30,48,11,2018,5
2,WOMEN LOSE WEIGHT (FEAT: SLICK RICK),257.41016,20181130_011248,1,30,48,11,2018,5
3,WON'T GO HOME WITHOUT YOU,231.23546,20181130_011705,1,30,48,11,2018,5
4,HEY_ SOUL SISTER,216.76363,20181130_012056,1,30,48,11,2018,5


In [44]:
"""try: 
    cur.execute("DROP table {}".format(time_table))
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)"""

try: 
    cur.execute(time_table_create)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

In [45]:
time_df.head()

,song,length,start_time,hour,day,week_of_year,month,year,weekday
0,JIM HENSON'S DEAD,182.85669,20181130_002207,0,30,48,11,2018,5
1,SHELL SHOCK,247.56200,20181130_010841,1,30,48,11,2018,5
2,WOMEN LOSE WEIGHT (FEAT: SLICK RICK),257.41016,20181130_011248,1,30,48,11,2018,5
3,WON'T GO HOME WITHOUT YOU,231.23546,20181130_011705,1,30,48,11,2018,5
4,HEY_ SOUL SISTER,216.76363,20181130_012056,1,30,48,11,2018,5


In [46]:
for row in time_df.itertuples(index=False):
    #print((time_table_insert, tuple(list(row))))
    cur.execute(time_table_insert,tuple(list(row)))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [47]:
user_df = df_next_song[["userId","firstName", "lastName","gender","level"]]

In [48]:
user_df.head(2)

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid


In [49]:
try: 
    cur.execute(user_table_create)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [50]:
for row in user_df.itertuples(index=False):
    #print((user_table_insert, tuple(list(row))))
    cur.execute(user_table_insert,tuple(list(row)) )
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [51]:
df.head(2)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,CASUAL,218.93179,1,SOMZWCG12A8C13C480,I DIDN'T MEAN TO,0
1,ARNTLGG11E2835DDB9,NaN,,NaN,CLP,266.39628,1,SOUDSGM12AC9618304,INSATIABLE (INSTRUMENTAL VERSION),0


In [52]:
#artist_df.artist_name = artist_df.artist_name.str.upper()

In [53]:
log_df.artist = log_df.artist.str.upper()

In [54]:
log_df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,STEPHEN LYNCH,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,MANOWAR,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [55]:
#pd.merge(artist_df,log_df,left_on="artist_name",right_on="artist")

In [56]:
log_df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,STEPHEN LYNCH,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,MANOWAR,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [57]:
artist_df.head(2)

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,CASUAL,California - LA,NaN,NaN
1,ARNTLGG11E2835DDB9,CLP,,NaN,NaN


In [58]:
try:
    conn.close()
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()
except:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

In [59]:
"""cols_songplay = ["start_time", "user_id", "level", "song_id","artist_id", "session_id", "location", "user_agent"]
values = len(cols_songplay) * ["%s"]
songplay_table_insert = ("""INSERT INTO {table} ({columns}) \
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s) \
                 
""".format(table = songplay_table,columns=",".join(cols_songplay)))"""


SyntaxError: invalid syntax (<ipython-input-59-c5d5a5409b8b>, line 3)

In [60]:


#songplay_table_create = ("""CREATE TABLE IF NOT EXISTS {table_name} (start_time varchar, user_id int, level varchar, song_id varchar,
#artist_id varchar, session_id int, location varchar, user_agent varchar);
#""".format(table_name = songplay_table))


"""try: 
    cur.execute("DROP table {}".format(songplay_table))
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)"""

try: 
    cur.execute(songplay_table_create)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

In [61]:
song_select

'select song_id, artist_id from (SELECT  artist_id,  song_id,  duration,  title as song,  name as artist  from artists  INNER JOIN songs using(artist_id) ) A\nwhere artist = %s and song = %s\nand duration = %s\n'

In [70]:
a =pd.merge(artist_df,song_df, on= "artist_id").drop_duplicates()

In [71]:
a.head(20)

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude,song_id,title,year,duration
0,ARD7TVE1187B99BFB1,CASUAL,California - LA,NaN,NaN,SOMZWCG12A8C13C480,I DIDN'T MEAN TO,0,218.93179
2,ARD7TVE1187B99BFB1,CASUAL,California - LA,NaN,NaN,SOQLGFP12A58A7800E,OAKTOWN,0,259.44771
9,ARNTLGG11E2835DDB9,CLP,,NaN,NaN,SOUDSGM12AC9618304,INSATIABLE (INSTRUMENTAL VERSION),0,266.39628
10,ARNTLGG11E2835DDB9,CLP,,NaN,NaN,SOZQDIU12A58A7BCF6,SUPERCONFIDENTIAL,0,338.31138
13,AR8ZCNI1187B9A069B,PLANET P PROJECT,,NaN,NaN,SOIAZJW12AB01853F1,PINK WORLD,1984,269.81832
14,AR10USD1187B99F3F1,TWEETERFRIENDLY MUSIC,"Burlington, Ontario, Canada",NaN,NaN,SOHKNRJ12A6701D1F8,DROP OF RAIN,0,189.57016
15,ARMJAGH1187FB546F3,THE BOX TOPS,"Memphis, TN",35.14968,-90.04892,SOCIWDW12A8C13D406,SOUL DEEP,1969,148.03546
16,ARKFYS91187B98E58F,JEFF AND SHERI EASTER,,NaN,NaN,SOYMRWW12A6D4FAB14,THE MOON AND I (ORDINARY DAY ALBUM VERSION),0,267.70240
17,ARD0S291187B9B7BF5,RATED R,Ohio,NaN,NaN,SOMJBYD12A6D4F8557,KEEPIN IT REAL (SKIT),0,114.78159
18,ARGSJW91187B9B1D6B,JENNYANYKIND,North Carolina,35.21962,-80.01955,SOQHXMF12AB0182363,YOUNG BOY BLUES,0,218.77506


In [76]:
log_df.song = log_df.song.str.upper()

In [80]:
pd.merge(log_df,a, right_on= ["artist_name","title","duration"], left_on = ["artist", "song", "length"])

0    15
Name: userId, dtype: object

In [81]:
#INNER JOIN songplays B using(duration, song, artist)
for index,row in log_df.iterrows():
    #print(row["artist"])
    cur.execute(song_select, (row["artist"], row["song"],row["length"]))
    results = cur.fetchone()

    if results:
        songid, artistid = results
        #print(results)
    else:
        songid, artistid = None, None
    #print(row)
    # insert songplay record
    if results and row["userId"]!="":
        songplay_data = row["ts"],row["userId"],row["level"],songid, artistid, row["sessionId"], row["location"],row["userAgent"]
        cur.execute(songplay_table_insert, (songplay_data))
        conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.